In [2]:
import pandas as pd
from fredapi import Fred
import requests
from bs4 import BeautifulSoup
import os
from dotenv import load_dotenv


In [54]:
# Directorio actual del notboook
current_dir = os.getcwd()

# Sube un directorio
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))

# camino al archivo
env_path = os.path.join(parent_dir, 'claves.env')

# Load the .env file
load_dotenv(env_path)

clave_fred = os.getenv("FRED_API_KEY")
#Clave API de Fred
fred = Fred(api_key=clave_fred)


In [3]:
load_dotenv('claves.env')

False

In [28]:

#FORMATO DE FECHA : AAAA-MM-DD
fechaInicio = '2023-06-1'
fechaFin = '2025-08-4'
# Get series data with custom start and end dates and frequency
inflacionSerie = fred.get_series(
    'CORESTICKM159SFRBATL',
    observation_start= fechaInicio,  # fecha de inicializacion
    observation_end= fechaFin,    # fecha de finalizacion
    frequency='m'                    # Tipos de frecuencia: 'm', 'q', 'sa', 'a'
)

df = inflacionSerie.reset_index()
df.columns = ["Fecha","Inflacion"]


In [29]:
df

,Fecha,Inflacion
0,2023-06-01,5.588761
1,2023-07-01,5.402919
2,2023-08-01,5.169678
3,2023-09-01,4.955445
4,2023-10-01,4.882717
5,2023-11-01,4.691251
6,2023-12-01,4.557667
7,2024-01-01,4.601332
8,2024-02-01,4.400073
9,2024-03-01,4.502758


In [13]:
# URL del informe ISM - Manufactura
url = "https://www.ismworld.org/supply-management-news-and-reports/reports/ism-report-on-business/pmi/july/"

# Get the HTML content
headers = {
    'User-Agent': 'Mozilla/5.0'
}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

# Find all tables
tables = soup.find_all('table')


PMI_manufacture = pd.read_html(str(tables))[0]
PMI_manufacture = PMI_manufacture.iloc[:-3]
PMI_manufacture.head()

,Index,Series Index Jul,Series Index Jun,Percentage Point Change,Direction,Rate of Change,Trend* (Months)
0,Manufacturing PMI®,48.0,49.0,-1.0,Contracting,Faster,5
1,New Orders,47.1,46.4,+0.7,Contracting,Slower,6
2,Production,51.4,50.3,+1.1,Growing,Faster,2
3,Employment,43.4,45.0,-1.6,Contracting,Faster,6
4,Supplier Deliveries,49.3,54.2,-4.9,Faster,From Slower,1


In [14]:
type(PMI_manufacture)

pandas.core.frame.DataFrame

In [48]:
import os
import requests
import pandas as pd
import yfinance as yf
from dotenv import load_dotenv
from sqlalchemy import create_engine
from fredapi import Fred
import requests
from bs4 import BeautifulSoup

def get_PMI():

    # URL del informe ISM - Manufactura
    url = "https://www.ismworld.org/supply-management-news-and-reports/reports/ism-report-on-business/pmi/july/"

    # Scrapeo de la tabla de manufactura
    headers = {
        'User-Agent': 'Mozilla/5.0'
    }
    
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all tables
    tables = soup.find_all('table')


    PMI_manufacture = pd.read_html(str(tables))[0]
    PMI_manufacture = PMI_manufacture.iloc[:-3]

    # URL del informe ISM - Serivicio
    url = "https://www.ismworld.org/supply-management-news-and-reports/reports/ism-report-on-business/services/june/"

    # sScrapeo de la table de serivicios
    headers = {
        'User-Agent': 'Mozilla/5.0'
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all tables
    tables = soup.find_all('table')

    PMI_service = pd.read_html(str(tables))[0]
    PMI_service = PMI_service.iloc[:-3]
    PMI_service = PMI_service.iloc[:,:-4]
    PMI_service = PMI_service.drop(columns=["Unnamed: 0_level_0"])
    PMI_service = PMI_service["Services PMI®"]

    return PMI_manufacture,PMI_service
PMI1, PMI2 = get_PMI()

C:\Users\corpo_zu\AppData\Local\Temp\ipykernel_8820\1641486823.py:47: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  PMI_service = PMI_service.drop(columns=["Unnamed: 0_level_0"])


In [49]:
PMI1

,Index,Series Index Jul,Series Index Jun,Percentage Point Change,Direction,Rate of Change,Trend* (Months)
0,Manufacturing PMI®,48.0,49.0,-1.0,Contracting,Faster,5
1,New Orders,47.1,46.4,+0.7,Contracting,Slower,6
2,Production,51.4,50.3,+1.1,Growing,Faster,2
3,Employment,43.4,45.0,-1.6,Contracting,Faster,6
4,Supplier Deliveries,49.3,54.2,-4.9,Faster,From Slower,1
5,Inventories,48.9,49.2,-0.3,Contracting,Faster,3
6,Customers’ Inventories,45.7,46.7,-1.0,Too Low,Faster,10
7,Prices,64.8,69.7,-4.9,Increasing,Slower,10
8,Backlog of Orders,46.8,44.3,+2.5,Contracting,Slower,34
9,New Export Orders,46.1,46.3,-0.2,Contracting,Faster,5


In [50]:
PMI2

,Series Index Jun,Series Index May,Percent Point Change,Direction,Rate of Change
0,50.8,49.9,+0.9,Growing,From Contracting
1,54.2,50.0,+4.2,Growing,From Unchanged
2,51.3,46.4,+4.9,Growing,From Contracting
3,47.2,50.7,-3.5,Contracting,From Growing
4,50.3,52.5,-2.2,Slowing,Slower
5,52.7,49.7,+3.0,Growing,From Contracting
6,67.5,68.7,-1.2,Increasing,Slower
7,42.4,43.4,-1.0,Contracting,Faster
8,51.1,48.5,+2.6,Growing,From Contracting
9,51.7,48.2,+3.5,Growing,From Contracting


In [23]:
import yfinance as yf

ticker = yf.Ticker("AAPL")

# Basic EPS info
info = ticker.info
eps = info['trailingEps']
print(f"EPS: {eps}")




EPS: 6.59


In [ ]:
import time  

avg_eps = 0
eps_promedio = 0

# Scrape S&P 500 tickers from Wikipedia
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
sp500_table = pd.read_html(url)[0]
tickers = sp500_table['Symbol'].tolist()
tickers = tickers[:101]

# Vamos a trabajar con las cifras en millones para evitar errores numéricos
market_cap_total = 0

# Dataframe con market cap y earnings per share del S&P500
ESPyMC = pd.DataFrame(columns=['ticker', 'EPS', 'market cap'])

for ticker in tickers:
    try:
        stock = yf.Ticker(ticker)
        info = stock.info

        eps = info.get('trailingEps', None)
        market_cap = info.get('marketCap', None)

        if eps is not None and market_cap is not None:
            market_cap_milliones = market_cap / 1e6
            market_cap_total += market_cap_milliones
            eps_promedio += eps*market_cap_milliones

            fila_temp = pd.DataFrame([{
                'ticker': ticker,
                'EPS': eps,
                'market cap': market_cap_milliones
            }])
            ESPyMC = pd.concat([ESPyMC, fila_temp], ignore_index=True)
      

    except Exception as e:
        print(f"Error con {ticker}: {e}")
       
print(ESPyMC)
print(f"Market cap total (en millones): {market_cap_total}")

#Calculamos el EPS promedio pesado

eps_promedio = eps_promedio/market_cap_total



   ticker    EPS     market cap
0     MMM   7.20   79990.374400
1     AOS   3.59    9839.928320
2     ABT   7.96  227182.247936
3    ABBV   2.09  350607.081472
4     ACN  12.58  156899.786752
..    ...    ...            ...
94    CRL  -0.60    8186.113536
95   SCHW   3.72  174756.167680
96   CHTR  36.54   36526.485504
97    CVX   7.76  308959.444992
98    CMG   1.13   57503.772672

[99 rows x 3 columns]
Market cap total (en millones): 18427605.613568008


In [41]:
def get_CP(fechaInicio,fechaFin):
    fred = Fred(api_key='4bb55f7e31f80a5d4347af428f87f6ac')

    cp = fred.get_series(
    'CP',
    observation_start= fechaInicio,  # fecha de inicializacion
    observation_end= fechaFin,    # fecha de finalizacion
    frequency='q'                    # Tipos de frecuencia: 'm', 'q', 'sa', 'a'
    )

    df = cp.reset_index()
    df.columns = ["Fecha","SRG"]
    
    return df

df = get_CP("2024-04-01","2025-05-01")

In [37]:
fred = Fred(api_key='4bb55f7e31f80a5d4347af428f87f6ac')
cp = fred.get_series(
'CP',
observation_start= "2024-04-01",  # fecha de inicializacion
observation_end= "2025-05-01",    # fecha de finalizacion
frequency='q'                    # Tipos de frecuencia: 'm', 'q', 'sa', 'a'
)



In [10]:
# Define the ticker (e.g., Apple)
ticker = yf.Ticker("AAPL")

# Get market cap
market_cap = ticker.info.get('marketCap')

# Format it for readability (optional)
formatted_cap = f"${market_cap:,.0f}"
print(f"Apple's Market Cap: {formatted_cap}")

Apple's Market Cap: $3,028,018,790,400
